Parcial 1: Fundamentos de Aprendizaje de Maquina

Autor: William Andrés Gómez Roa

06/09/2024

# **MODELO NAIVE BAYES**

**Influencia de características demográficas sobre el ingreso**

**Objetivo:** Predicción del ingreso de una persona con base en variables categóricas

Dataset: Censo.csv (seleccione 10.000 registros aleatoriamente, tome solo las variables categóricas)

Descripciones de los campos:

1. **age**: Edad de la persona
2. workclass: Clase de trabajo que tiene la persona
3. **fnlwgt**: Indicador numérico de la proporción de la población que la persona representa
4. **education**: Nivel de educación
5. **education_num**: número de años de estudio
6. **marital_status**: Estado civil de la persona
7. **occupation**: Area en que la persona trabaja
8. **relationship**: Tipo de relación que tiene la persona
9. **race**: Raza de la persona
10. **sex**: Sexo de la persona
11. **capital_gain**: Ganancias de capital obtenidas
12. **capital_loss**: Pérdidas de capital
13. **hours_per_week**: Número medio de horas de trabajo a la semana
14. **native_country**: País de origen
15. ***salary***: Nivel de ingresos

**Preguntas:**

1. (12) Dentro de su modelo, ensaye 3 valores del umbral de probabilidad, para decidir si una persona tiene un nivel de ingreso menor o mayor a 50 mil dólares. ¿Cuál valor ofrece el mejor desempeño?

# **SOLUCION**

In [57]:
SEED <- 837

In [58]:
censo <- read.csv("/content/Censo.csv")
str(censo)

'data.frame':	48842 obs. of  15 variables:
 $ age            : int  25 38 28 44 18 34 29 63 24 55 ...
 $ workclass      : chr  "Private" "Private" "Local-gov" "Private" ...
 $ fnlwgt         : int  226802 89814 336951 160323 103497 198693 227026 104626 369667 104996 ...
 $ education      : chr  "11th" "HS-grad" "Assoc-acdm" "Some-college" ...
 $ educational.num: int  7 9 12 10 10 6 9 15 10 4 ...
 $ marital.status : chr  "Never-married" "Married-civ-spouse" "Married-civ-spouse" "Married-civ-spouse" ...
 $ occupation     : chr  "Machine-op-inspct" "Farming-fishing" "Protective-serv" "Machine-op-inspct" ...
 $ relationship   : chr  "Own-child" "Husband" "Husband" "Husband" ...
 $ race           : chr  "Black" "White" "White" "Black" ...
 $ gender         : chr  "Male" "Male" "Male" "Male" ...
 $ capital.gain   : int  0 0 0 7688 0 0 0 3103 0 0 ...
 $ capital.loss   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ hours.per.week : int  40 50 40 40 30 30 40 32 40 10 ...
 $ native.country : chr  "United-Sta

In [59]:
# seleccione 10.000 registros aleatoriamente, tome solo las variables categóricas
set.seed(SEED)
censo <- censo[, c("workclass", "education", "marital.status", "occupation",
             "relationship", "race", "gender", "native.country", "income")]
censo <- censo[sample(nrow(censo), 10000), ]
str(censo)

'data.frame':	10000 obs. of  9 variables:
 $ workclass     : chr  "Private" "Private" "Private" "Self-emp-inc" ...
 $ education     : chr  "Bachelors" "Bachelors" "HS-grad" "Some-college" ...
 $ marital.status: chr  "Never-married" "Married-civ-spouse" "Married-civ-spouse" "Married-civ-spouse" ...
 $ occupation    : chr  "Prof-specialty" "Prof-specialty" "Handlers-cleaners" "Exec-managerial" ...
 $ relationship  : chr  "Not-in-family" "Husband" "Husband" "Husband" ...
 $ race          : chr  "White" "White" "White" "White" ...
 $ gender        : chr  "Male" "Male" "Male" "Male" ...
 $ native.country: chr  "United-States" "United-States" "United-States" "United-States" ...
 $ income        : chr  "<=50K" ">50K" "<=50K" ">50K" ...


In [60]:
#Entrenamiento y Prueba para censo
set.seed(SEED)
sample <- sample.int(nrow(censo), floor(.75*nrow(censo)))
censo.train <- censo[sample, ]
censo.test <- censo[-sample, ]

## MODELO NAIVE-BAYES

In [61]:
if (!require('e1071')) install.packages('e1071')
library(e1071)

In [62]:
naivetest<-naiveBayes(censo.train$income~.,censo.train)

In [63]:
#tasas base
naivetest$apriori

Y
<=50K  >50K 
 5748  1752 

In [64]:
naivetest$tables$gender

       gender
Y          Female      Male
  <=50K 0.3856994 0.6143006
  >50K  0.1335616 0.8664384

## EVALUANDO EL MODELO

In [65]:
probabilidades <- predict(naivetest, censo.test, type="raw")
dim(probabilidades)

[1] 2500    2

A continuación se presenta una función que permite evaluar las probilidades obtenidas con 'predict' utilizando el modelo entrenado y los datos de prueba. Esta funcion utiliza el valor de **censo.test$income** para verificar si la prediccion fue correcta o no, según el umbral escogido.

Se calculan las métricas de evaluación: TP, FP, TN, FN, para posteriormente calcular el indice de la 'Precision' y poder evlauar los umbrales seleccionados.

In [66]:
contar_clases_por_umbral <- function(probabilidades, censo_test_income, umbral) {
  # Inicializar contadores
  contador_clase_1 <- 0
  contador_clase_2 <- 0
  TP <- 0
  FP <- 0
  TN <- 0
  FN <- 0

  # Convertir income en factor para comparación
  clase_real <- ifelse(censo_test_income == ">50K", ">50K", "<=50K")

  # Inicializar vectores para guardar clasificaciones y si son correctas
  clasificaciones <- character(length(censo_test_income))

  # Recorrer las probabilidades y contar las frecuencias para el umbral dado
  for (i in 1:length(censo_test_income)) {
    if (probabilidades[i, 2] > umbral) {
      clasificaciones[i] <- ">50K"
      contador_clase_2 <- contador_clase_2 + 1
    } else {
      clasificaciones[i] <- "<=50K"
      contador_clase_1 <- contador_clase_1 + 1
    }

    # Actualizar contadores de TP, FP, TN, FN
    if (clasificaciones[i] == ">50K" && clase_real[i] == ">50K") {
      TP <- TP + 1
    } else if (clasificaciones[i] == ">50K" && clase_real[i] == "<=50K") {
      FP <- FP + 1
    } else if (clasificaciones[i] == "<=50K" && clase_real[i] == "<=50K") {
      TN <- TN + 1
    } else if (clasificaciones[i] == "<=50K" && clase_real[i] == ">50K") {
      FN <- FN + 1
    }
  }

  # Calcular precisión
  precision <- ifelse((TP + FP) == 0, NA, TP / (TP + FP))

  # Crear un dataframe con las frecuencias
  frecuencias <- data.frame(
    Clase = c("<=50K", ">50K"),
    Frecuencia = c(contador_clase_1, contador_clase_2)
  )

  # Crear un dataframe con las clasificaciones
  resultados <- data.frame(
    Clasificacion = clasificaciones,
    Real = clase_real
  )

  return(list(Frecuencias = frecuencias, Resultados = resultados, Precision = precision))
}

**UMBRAL: 0.4**

In [67]:
umbral <- 0.4
resultado <- contar_clases_por_umbral(probabilidades, censo.test$income, umbral)
# Mostrar frecuencias y precisión
print(resultado$Frecuencias)
print(paste("Precisión:", resultado$Precision))

  Clase Frecuencia
1 <=50K       1633
2  >50K        867
[1] "Precisión: 0.521337946943483"


**UMBRAL: 0.5**

In [68]:
umbral <- 0.5
resultado <- contar_clases_por_umbral(probabilidades, censo.test$income, umbral)

# Mostrar frecuencias y precisión
print(resultado$Frecuencias)
print(paste("Precisión:", resultado$Precision))

  Clase Frecuencia
1 <=50K       1728
2  >50K        772
[1] "Precisión: 0.555699481865285"


**UMBRAL: 0.8**

In [69]:
umbral <- 0.8
resultado <- contar_clases_por_umbral(probabilidades, censo.test$income, umbral)

# Mostrar frecuencias y precisión
print(resultado$Frecuencias)
print(paste("Precisión:", resultado$Precision))

  Clase Frecuencia
1 <=50K       2076
2  >50K        424
[1] "Precisión: 0.683962264150943"


# **RESPUESTA**

EL valor que ofrece el mejor desempeño es el umbral de 0.8 utilizando la metrica de 'Presicion'.